In [2]:
import csv
import os
import re
from glob import glob
from itertools import product
from math import atan2, degrees, sqrt
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal, stats
from scipy.fft import fft, ifft
from scipy.interpolate import interp1d, make_interp_spline
from scipy.spatial.distance import euclidean
from sklearn.preprocessing import MinMaxScaler

pd.set_option("display.max_columns", 500)
sns.set_style("darkgrid", rc={"xtick.bottom": True, "ytick.left": True})

# Data Input and Cleaning Functions

## DataFrame Functions

In [3]:
def read_raw_data(path: str) -> pd.DataFrame:
    return pd.read_csv(
        path,
        skiprows=[0],
        header=[0, 1, 2],
        index_col=[0],
    )


def clean_data(raw_df: pd.DataFrame) -> pd.DataFrame:
    def combine_levels(df: pd.DataFrame) -> pd.DataFrame:
        df.columns = df.columns.map("_".join)
        return df

    def drop_likelihood(df: pd.DataFrame) -> pd.DataFrame:
        mask = df.columns.str.contains("likelihood")
        cols_to_drop = df.columns[mask]
        return df.drop(columns=cols_to_drop)

    return (
        raw_df.rename_axis(index="frame")
        .interpolate(method="linear")
        .pipe(combine_levels)
        .pipe(drop_likelihood)
    )

## Path Functions

In [4]:
def get_mouse_name(path):
    """Extract mouse name from a given path."""

    pattern = r"[\\/]?(([^\\/]+))DLC"

    if match := re.search(pattern, path):
        return match.group(1)
    else:
        print("No match found")

In [11]:
def get_files():
    pattern = r"(\d+\w+)_(Sham|Ctl|SCI|CCI|Baseline|EAE|mtPst1)_(-?\d+)dpi"
    paths = glob("*DLC*.csv")
    file_info = []

    for path in paths:
        if match := re.search(pattern, path):
            animal_id, condition, dpi = match.groups()
            file_info.append({"file_name": path, "animal_id": animal_id, "condition": condition, "dpi": int(dpi)})

    sorted_files = sorted(file_info, key=lambda x: (x["dpi"], x["condition"]))

    return [info["file_name"] for info in sorted_files]

# Metric Functions

In [5]:
def euclidian_dist(row: pd.Series, *, col_root_pair: tuple[str]) -> pd.Series:
    """Calculate the 2d euclidean distance between two body parts, using a DataFrame
    containing columns representing individual x and y coordinates.

    Parameters
    ----------
    row : pd.Series
        pandas series representing a single observation
    col_root_pair : tuple[str]
        containing pair of body parts representing the root body part of an x,y pair.
        e.g., ('Mouse_Snout', 'Mouse_Tail_2base')

    Returns
    -------
    pd.Series
        euclidean distance for the row and body part pair in the DataFrame
    """

    return euclidean(
        (row[f"{col_root_pair[0]}_x"], row[f"{col_root_pair[0]}_y"]),
        (row[f"{col_root_pair[1]}_x"], row[f"{col_root_pair[1]}_y"]),
    )

# Categorizing Frames into Inner or Outer Box

In [12]:
def distance(x1, y1, x2, y2): 
    result = np.sqrt(
        np.square(x2 - x1) + np.square(y2 - y1)
    )
    
    return result

paths = get_files()

total_distance_results = []
Total_Distances = []

for path in paths:
    mouse_name = get_mouse_name(path)
    raw_df = read_raw_data(path)
    cleaned_df = clean_data(raw_df)
    # Calculate total distance using the TotDist function
    relevant_data = cleaned_df[["Mouse_Tail_2base_x", "Mouse_Tail_2base_y"]]
    newrows = (
        relevant_data.assign(
            new_frame_x=lambda df: df["Mouse_Tail_2base_x"].shift(-1),
            new_frame_y=lambda df: df["Mouse_Tail_2base_y"].shift(-1),
            Distance=lambda df: distance(
                df["Mouse_Tail_2base_x"],
                df["Mouse_Tail_2base_y"],
                df["new_frame_x"],
                df["new_frame_y"],
            ) / 35,
    ))
    total_distance_results.append(newrows)   
    Travel = newrows.iloc[:7200, -1:] 
    # Rename the 'Distance' column with the mouse name
    Travel.rename(columns={'Distance': f'{mouse_name}'}, inplace=True) 
    Total_Distances.append(Travel.sum(axis=0)) 

#  Concatenate the total distance results into a single DataFrame
total_distance_series = pd.concat(Total_Distances, axis=0)
total_distance_df = total_distance_series.to_frame()
total_distance_df.columns = ['Total Distance (cm)']
########################################################################################
total_distance_df.to_excel('Total Distance Traveled EAE All.xlsx')

In [8]:
XMIN = 357
XMAX = 1428
YMIN = 357
YMAX = 1428
YDIV = 892.5
XDIV = 892.5

paths = get_files()

box_count_dfs = []
total_distance_results = []
for path in paths:
    mouse_name = get_mouse_name(path)
    raw_df = read_raw_data(path)
    cleaned_df = clean_data(raw_df)
    box_count = (
        cleaned_df.assign(
            box=lambda df: np.select(
                [
                    (df["Mouse_Tail_2base_x"] > XMIN)
                    & (df["Mouse_Tail_2base_x"] < XMAX)
                    & (df["Mouse_Tail_2base_y"] > YMIN)
                    & (df["Mouse_Tail_2base_y"] < YMAX),
                    (df["Mouse_Tail_2base_x"] <= XDIV)
                    & (df["Mouse_Tail_2base_y"] <= YDIV),
                    (df["Mouse_Tail_2base_x"] > XDIV)
                    & (df["Mouse_Tail_2base_y"] <= YDIV),
                    (df["Mouse_Tail_2base_x"] <= XDIV)
                    & (df["Mouse_Tail_2base_y"] > YDIV),
                    (df["Mouse_Tail_2base_x"] > XDIV)
                    & (df["Mouse_Tail_2base_y"] > YDIV),
                ],
                ["inner", "OuterBox_TL", "OuterBox_TR", "OuterBox_BL", "OuterBox_BR"],
                default="outer",
            )
        )["box"]
        .value_counts()
        .div(60)
        .to_frame(name=mouse_name)
        .transpose()
        .rename_axis(columns=None)
    )
    box_count_dfs.append(box_count)

grouped_box_counts = pd.concat(box_count_dfs).T
########################################################################################
grouped_box_counts.to_excel('Open Field Analysis EAE All.xlsx')